In [1]:
import pandas as pd
from zhon.hanzi import punctuation
import re

In [2]:
def full_to_half(s):
    # 將字符串 s 中的全形字符和標點符號轉換為半形。
    n = []
    for char in s:
        code = ord(char)
        if code == 0x3000:  # 全形空格直接轉換
            code = 32
        elif 0xFF01 <= code <= 0xFF5E:  # 全形字符（除空格）轉換成半形
            code -= 0xFEE0
        n.append(chr(code))
    return ''.join(n)

In [3]:
with open('./Data/stopwords_tc.txt', encoding='utf-8', mode='r') as f:
    stop_words = []
    for l in f:
        stop_words.append(l.strip())

In [4]:
ptt_food_post_df = pd.read_csv('./Data/Ptt/ptt_food_post_list.csv', index_col='Pid')
ptt_food_post_df

,title,link,date,content
Pid,,,,
0,[食記] 台南 來一片。一周只賣三天的美味披蕯店,https://www.ptt.cc/bbs/Food/M.1669640419.A.EFD...,2022/11/28,餐廳名稱：來一片消費時間：2022年/10月電話：0905-539108地址：台南市仁德區二...
1,[食記] 台北 辰壽司割烹~大閘蟹宴,https://www.ptt.cc/bbs/Food/M.1669642084.A.CE7...,2022/11/28,餐廳名稱：辰壽司割烹 消費時間：2022年/11月 地址：台北市松山區敦化北路...
2,[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶,https://www.ptt.cc/bbs/Food/M.1669646872.A.E61...,2022/11/28,鍋物最前線--金大鋤壽喜燒烤鍋物新莊店下午茶消費日期：2022 年 11 月有圖有真相圖文並...
3,[食記] 新北林口｜林口三井的點點心,https://www.ptt.cc/bbs/Food/M.1669680859.A.700...,2022/11/29,餐廳名稱：林口三井OUTLET|放鬆心情 來吃港式的點點心 消費時間：2022/1...
4,[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯,https://www.ptt.cc/bbs/Food/M.1669706692.A.EFC...,2022/11/29,餐廳名稱：夯・魯肉飯 消費時間：2022年/10月110台北市信義區松山路152號...
...,...,...,...,...
1002,[食記] 台北 中山區 欣葉台菜創始店,https://www.ptt.cc/bbs/Food/M.1664473643.A.7CA...,2022/09/30,餐廳名稱： 欣葉台菜創始店消費時間：2022年9月地址：台北市中山區雙城街34之1號營業時間...
1003,[食記] 台北信義-寓所咖啡Utroo cafe-咖啡廳,https://www.ptt.cc/bbs/Food/M.1664505703.A.CC6...,2022/09/30,餐廳名稱 : 寓所咖啡 Utroo café消費時間 : 2022年/8月地址 : ...
1004,[食記] 屏東潮州-牛大福。屏東牛肉料理,https://www.ptt.cc/bbs/Food/M.1664507458.A.84A...,2022/09/30,餐廳名稱：屏東。潮州》牛大福。屏東牛肉料理 消費時間：2022年/8月 地址：...


In [5]:
test_content = ptt_food_post_df.loc[0, 'content']
test_content

'餐廳名稱：來一片消費時間：2022年/10月電話：0905-539108地址：台南市仁德區二空路82號營業時間：五~日 11:30~14:30 17:30~21:00blog圖文版：https://morrisyu.com/archives/6887來一片，位在台南二空眷村的一家低調披蕯店，原本平日賣鍋假日賣披蕯，現在一周營業三天主打披蕯，在其他的分享文章中發現，來一片主廚曾在台中小鍋牛、台北Solo Pasta服務過，主廚也得過APN創意披蕯獎項。來一片店址位在國宅區內，沒有華麗的裝潢，內用座位像是一般的小餐館一樣，桌數不多，想要來用餐建議一定要訂位，雖然不在市區，不過生意真的蠻不錯的。menu，餐點內容每周都會調整，不得不說，來一片的價格真的很平實，主打Pizza，另外還有開胃小點及義大利麵。蝦膏濃湯，附上一片麵包及小明蝦，湯頭真的是滿滿的蝦味，又濃又好喝。松露炸薯條，薯修炸得微酥，灑上滿滿的松露提味。瑪格利特披薩，不得不說這樣一個140元真的很便宜，上桌後就聞到滿滿的香氣，餅皮微酥愈嚼香愈香，搭上蕃茄、起司、羅勒帶出的酸甜風味，雖然是基本款的披蕯，真的是超美味。拿坡里簡單炸海鮮，有蝦子、丁香魚、魚肉片、中卷等，炸得微酥不油膩，讓人滿足。茄汁甜椒培根義大利麵，麵條吃起來相當Q彈有嚼勁，茄汁的甜感度十足。榛果巧克力醬炸披薩條，披蕯皮炸酥後再淋上巧克力醬，披蕯皮炸後酥軟像是在吃甜不辣一樣，淋上巧克力醬後甜甜的，整盤吃完熱量爆表。            @__           _,>/-_   攝影‧旅行‧拈花惹草→Morris          (_) \\(_)________ http://morrisyu.com/'

In [6]:
def remove_urls_and_phones(text):
    """
    移除文字中的網址和電話號碼。
    """
    # 正則表達式匹配網址
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_pattern, '', text)

    # 正則表達式匹配電話號碼（適用於多種常見格式）
    phone_pattern = r'(\d{2,4}[-.\s]??\d{3,4}[-.\s]??\d{3,4}|\(\d{2,4}\)\s*\d{3,4}[-.\s]??\d{3,4}|\d{10,11})'
    text = re.sub(phone_pattern, '', text)

    return text

In [7]:
def remove_english(text):
    """
    移除文字中的所有英文字符。
    """
    # 正則表達式匹配所有英文字母和英文單詞
    pattern = r'[A-Za-z]+'
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

In [8]:
def load_dictionary(file_path):
    with open(file_path, encoding='utf-8', mode='r') as f:
        dictionary = [l.strip() for l in f]
    return dictionary

In [9]:
def remove_stopwords(text, stopwords):
    # 使用列表推導式過濾掉停用詞
    filtered_words = remove_urls_and_phones(text)
    filtered_words = remove_english(filtered_words)
    filtered_words = "".join(c for c in filtered_words if c not in ('；','，','。','！','：','「','」','…','、','？','【','】','.',':','?',';','!','~','`','+','-','<','>','/','[',']','{','}',"'",'"','\\', ' ', '‧'))
    filtered_words = "".join(word for word in filtered_words if word not in stopwords)
    # 將過濾後的單詞列表重新組合成字符串
    return filtered_words

In [10]:
stop_words = load_dictionary('./Data/stopwords_tc.txt')
test_content = remove_stopwords(test_content, stop_words)
test_content

'餐廳名稱片消費時間年月電話址台南市仁德區空路號營業時間日圖文版片位台南空眷村家低調披蕯店原平日賣鍋假日賣披蕯現周營業天主披蕯分享文章中發現片主廚台中鍋牛台北服務主廚創意披蕯獎項片店址位國宅區華麗裝潢座位般餐館樣桌數想餐建議訂位市區生意真錯餐點容周調整說片價格真平實主外開胃點義利麵蝦膏濃湯附片麵包明蝦湯頭真蝦味濃喝松露炸薯條薯修炸微酥灑松露提味瑪格利特披薩說樣元真宜桌聞香氣餅皮微酥愈嚼香愈香搭蕃茄司羅勒酸甜風味基款披蕯真超美味坡里簡炸海鮮蝦子丁香魚魚肉片中卷炸微酥油膩足茄汁甜椒培根義利麵麵條吃相彈嚼勁茄汁甜感度十足榛果巧克力醬炸披薩條披蕯皮炸酥淋巧克力醬披蕯皮炸酥軟吃甜辣樣淋巧克力醬甜甜整盤吃完熱量爆表攝影旅行拈花惹草'

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba

def tfidf_vectorlize(text):
    # 使用 jieba 進行分詞
    seg_list = jieba.cut(text)

    # 將分詞結果轉化為空格分隔的字符串
    text = ' '.join(seg_list)

    # 初始化 TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # 轉換文本
    tfidf_vector = vectorizer.fit_transform([text])

    # 查看得到的 TF-IDF 向量
    return tfidf_vector.toarray()

In [12]:
tfidf_vectorlize(test_content)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/rh/7g0n3djn3wj8dlnjv7plmzcr0000gn/T/jieba.cache
Loading model cost 0.143 seconds.
Prefix dict has been built succesfully.


array([[0.08421519, 0.08421519, 0.08421519, 0.16843038, 0.08421519,
        0.08421519, 0.08421519, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.08421519, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.08421519, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.08421519, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.08421519, 0.08421519, 0.08421519, 0.25264558,
        0.08421519, 0.08421519, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.08421519, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.16843038, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.08421519, 0.08421519, 0.08421519, 0.16843038,
        0.08421519, 0.16843038, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.08421519, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.08421519, 0.16843038, 0.08421519, 0.08421519,
        0.16843038, 0.08421519, 0.08421519, 0.08421519, 0.08421519,
        0.08421519, 0.08421519, 0.08421519, 0.08

In [13]:
import gensim
from gensim.models.word2vec import Word2Vec

import numpy as np

def word2vec_vectorlize(text):
    # 加载预训练的 Word2Vec 模型
    model = gensim.models.KeyedVectors.load_word2vec_format('./IgnoreFile/tmunlp_1.6B_WB_50dim_2020v1.bin.gz', 
                                                        unicode_errors='ignore', 
                                                        binary=True)
    
    words = text.split()  # 假设文本已经分词
    vector_size = model.vector_size
    sentence_vector = np.zeros(vector_size)
    
    count = 0
    for word in words:
        if word in model:
            sentence_vector += model[word]
            count += 1
    if count > 0:
        sentence_vector /= count
    
    return sentence_vector

In [14]:
word2vec_vectorlize(test_content)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
from transformers import BertTokenizer, BertModel
import torch

def bert_vectorlize(text):
    # 初始化 BERT 分詞器和模型
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    model = BertModel.from_pretrained('bert-base-chinese')

    # 使用 jieba 分詞後，將分詞結果合併為一個字符串
    seg_list = jieba.cut(text)
    text = ' '.join(seg_list)

    # 使用 BERT 分詞器處理文本
    inputs = tokenizer(text, return_tensors="pt")

    # 獲取 BERT 的輸出
    with torch.no_grad():
        outputs = model(**inputs)

    # 獲取句子的 BERT 向量（例如取平均）
    sentence_embedding = outputs.last_hidden_state.mean(dim=1)
    return sentence_embedding

/Users/jim/anaconda3/envs/forPttCrawler/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
bert_vectorlize(test_content)

/Users/jim/anaconda3/envs/forPttCrawler/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tensor([[ 4.8229e-01,  2.5667e-03, -5.8856e-01,  2.1391e-01, -1.6442e-01,
         -5.4613e-01,  3.5924e-01,  4.6762e-01, -5.5558e-01, -3.5682e-01,
         -7.7464e-02, -8.2425e-01,  1.7101e-01,  3.9789e-01,  3.7753e-01,
         -1.6625e-02,  2.5990e-01, -4.1578e-02,  8.2027e-01, -1.9985e-01,
         -3.5578e-01,  5.9739e-01,  2.7625e-02,  2.6349e-03,  2.1128e-01,
          1.8082e-01,  2.6003e-01, -6.6504e-01,  2.3391e-01, -1.5353e-01,
         -9.8917e-02, -4.9456e-01,  2.5902e-01,  2.2306e-01, -1.4745e-01,
         -3.6580e-01,  4.8630e-02,  3.9288e-01, -4.3881e-01, -7.9251e-02,
          1.8907e-01,  8.3506e-02, -6.6132e-01, -8.0387e-02,  5.3433e-01,
          6.0422e-02,  2.0127e-01,  3.3371e-02,  1.4884e-01,  2.2203e-02,
          2.9872e-02,  9.3898e+00, -6.0889e-03,  1.3550e-01, -4.2906e-01,
          4.8790e-01,  3.4051e-01, -2.8113e-01,  1.8711e-01, -3.4987e-01,
          1.7059e-03,  4.4931e-02, -5.8915e-01,  2.7125e-01, -1.2531e-01,
          1.0672e-01,  1.7819e-01,  1.